In [2]:
import pandas as pd
import geopandas as gp

In [4]:
#Loading Census data
data_census = pd.read_csv('Census_2010.csv')

In [5]:
data_census.head()

,Zipcode,Sex,Total,White,Black,Asian,Hispanic,Mixed Non-Hispanic
0,10001,Female,10918,5929,745,1987,1913,265
1,10001,Male,10184,5775,857,1532,1735,231
2,10002,Female,42840,8032,2906,19675,11472,624
3,10002,Male,38570,8298,2351,18064,9147,576
4,10003,Female,29274,20677,788,4620,2223,825


In [23]:
#Renaming ZipCode column
data_census = data_census.rename(columns = {'Zipcode':'ZipCode'})

In [6]:
#Loading 311 data
data_311 = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv')
data_311 = data_311.rename(columns = {'Incident Zip':'ZipCode'})

In [35]:
data_311.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,ZipCode,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Latitude,Longitude,Location
0,19503049,12/31/2010 11:59:12 PM,01/01/2011 02:14:44 AM,NYPD,New York City Police Department,Blocked Driveway,11212,BROOKLYN,1008500,181726,40.665439,-73.912589,"(40.665439409429055, -73.91258883811614)"
1,19503621,12/31/2010 11:57:30 PM,01/01/2011 12:46:41 AM,NYPD,New York City Police Department,Blocked Driveway,11365,QUEENS,1037441,205363,40.730191,-73.808082,"(40.73019074217415, -73.80808214665396)"
2,19504838,12/31/2010 11:54:59 PM,01/01/2011 08:03:26 AM,NYPD,New York City Police Department,Blocked Driveway,11236,BROOKLYN,1006381,176021,40.649786,-73.920246,"(40.64978599344521, -73.9202456701406)"
3,19506107,12/31/2010 11:54:00 PM,03/16/2011 09:36:00 AM,DOT,Department of Transportation,Street Light Condition,10006,MANHATTAN,980807,197440,40.708603,-74.012419,"(40.70860327698897, -74.01241861690464)"
4,19507090,12/31/2010 11:52:00 PM,02/01/2011 12:45:00 PM,DEP,Department of Environmental Protection,Noise,11209,BROOKLYN,978741,169492,40.631891,-74.019848,"(40.63189122768689, -74.01984766770063)"


In [9]:
#Loading MapPLUTO data
data_MNpluto = data = gp.GeoDataFrame.from_file("Manhattan/MNMapPLUTO.shp")
data_BKpluto = data = gp.GeoDataFrame.from_file("Brooklyn/BKMapPLUTO.shp")

In [13]:
#Concatenating Manhattan and Brooklyn
data_mappluto =  gp.GeoDataFrame(pd.concat([data_MNpluto, data_BKpluto], ignore_index=True))

In [18]:
data_BKpluto.head()

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0,None,HUDSON AVENUE,M3-1,None,7,11250,11250,3000060010,0,...,0,0,None,11201,M3-1,None,None,None,12d,"POLYGON ((989159.7080000043 196397.7844000012,..."
1,0,None,64 JOHN STREET,M1-4/R7A/MX-2,None,7,241200,1296000,3000200014,33575,...,0,1900,None,11201,M1-4/R7A,None,None,None,12d,"POLYGON ((988353.2639999986 195869.5602000058,..."
2,0,None,23 BRIDGE STREET,M3-1,None,7,21600,26550,3000210007,0,...,0,1920,None,11201,M3-1,None,None,None,12d,"POLYGON ((988650.2011999935 195785.6609999985,..."
3,0,None,49 HUDSON AVENUE,R6B,None,7,4542,24468,3000340006,2484,...,0,1920,None,11201,R6B,None,None,None,12d,"POLYGON ((989597.2610000074 195585.5018000007,..."
4,0,None,71 GOLD STREET,R6B,None,7,8502,26872,3000430009,3252,...,0,1925,None,11201,R6B,None,None,None,12d,"POLYGON ((989141.1386000067 195346.7888000011,..."


In [79]:
#Grouping MapPLUTO data by ZIP code and summing
res_mappluto = pd.DataFrame(data_mappluto['UnitsRes'].groupby(data_mappluto['ZipCode']).sum())
res_mappluto['ZipCode'] = res_mappluto.index
res_mappluto.index = range(0,len(res_mappluto))
res_mappluto.dtypes

UnitsRes    int64
ZipCode     int64
dtype: object

In [77]:
#Combining genders for Census data
grouped_census = data_census.groupby(['ZipCode']).sum()
grouped_census['ZipCode'] = grouped_census.index
grouped_census.index = range(0,len(grouped_census))
grouped_census.head()

,Total,White,Black,Asian,Hispanic,Mixed Non-Hispanic,ZipCode
0,21102,11704,1602,3519,3648,496,10001
1,81410,16330,5257,37739,20619,1200,10002
2,56024,40056,1738,8020,4489,1437,10003
3,3089,2034,86,653,189,108,10004
4,7135,4663,229,1472,530,196,10005


In [78]:
#Joining Census data to 311 data
data_311census = pd.merge(data_311, grouped_census, how='left', on=['ZipCode'])
data_311census.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,ZipCode,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Latitude,Longitude,Location,Total,White,Black,Asian,Hispanic,Mixed Non-Hispanic
0,19503049,12/31/2010 11:59:12 PM,01/01/2011 02:14:44 AM,NYPD,New York City Police Department,Blocked Driveway,11212,BROOKLYN,1008500,181726,40.665439,-73.912589,"(40.665439409429055, -73.91258883811614)",84500,666,67801,588,13905,1034
1,19503621,12/31/2010 11:57:30 PM,01/01/2011 12:46:41 AM,NYPD,New York City Police Department,Blocked Driveway,11365,QUEENS,1037441,205363,40.730191,-73.808082,"(40.73019074217415, -73.80808214665396)",42252,14064,3421,17532,6094,930
2,19504838,12/31/2010 11:54:59 PM,01/01/2011 08:03:26 AM,NYPD,New York City Police Department,Blocked Driveway,11236,BROOKLYN,1006381,176021,40.649786,-73.920246,"(40.64978599344521, -73.9202456701406)",93877,5066,77175,2319,7358,1396
3,19506107,12/31/2010 11:54:00 PM,03/16/2011 09:36:00 AM,DOT,Department of Transportation,Street Light Condition,10006,MANHATTAN,980807,197440,40.708603,-74.012419,"(40.70860327698897, -74.01241861690464)",3011,1947,135,590,248,83
4,19507090,12/31/2010 11:52:00 PM,02/01/2011 12:45:00 PM,DEP,Department of Environmental Protection,Noise,11209,BROOKLYN,978741,169492,40.631891,-74.019848,"(40.63189122768689, -74.01984766770063)",68853,47714,1173,7889,10322,1419


In [80]:
#Joining residential units to 311 data
data_311census['ZipCode'] = data['ZipCode'].convert_objects(convert_numeric=True)

In [82]:
data_311censusres = pd.merge(data_311census, res_mappluto, how='left', on=['ZipCode'])
data_311censusres.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,ZipCode,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Latitude,Longitude,Location,Total,White,Black,Asian,Hispanic,Mixed Non-Hispanic,UnitsRes
0,19503049,12/31/2010 11:59:12 PM,01/01/2011 02:14:44 AM,NYPD,New York City Police Department,Blocked Driveway,11201,BROOKLYN,1008500,181726,40.665439,-73.912589,"(40.665439409429055, -73.91258883811614)",84500,666,67801,588,13905,1034,34398
1,19503621,12/31/2010 11:57:30 PM,01/01/2011 12:46:41 AM,NYPD,New York City Police Department,Blocked Driveway,11201,QUEENS,1037441,205363,40.730191,-73.808082,"(40.73019074217415, -73.80808214665396)",42252,14064,3421,17532,6094,930,34398
2,19504838,12/31/2010 11:54:59 PM,01/01/2011 08:03:26 AM,NYPD,New York City Police Department,Blocked Driveway,11201,BROOKLYN,1006381,176021,40.649786,-73.920246,"(40.64978599344521, -73.9202456701406)",93877,5066,77175,2319,7358,1396,34398
3,19506107,12/31/2010 11:54:00 PM,03/16/2011 09:36:00 AM,DOT,Department of Transportation,Street Light Condition,11201,MANHATTAN,980807,197440,40.708603,-74.012419,"(40.70860327698897, -74.01241861690464)",3011,1947,135,590,248,83,34398
4,19507090,12/31/2010 11:52:00 PM,02/01/2011 12:45:00 PM,DEP,Department of Environmental Protection,Noise,11201,BROOKLYN,978741,169492,40.631891,-74.019848,"(40.63189122768689, -74.01984766770063)",68853,47714,1173,7889,10322,1419,34398


In [84]:
data_311censusres['Density'] = data_311censusres['UnitsRes']/data_311censusres['Total']*1000

In [86]:
data_311censusres.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,ZipCode,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),...,Longitude,Location,Total,White,Black,Asian,Hispanic,Mixed Non-Hispanic,UnitsRes,Density
0,19503049,12/31/2010 11:59:12 PM,01/01/2011 02:14:44 AM,NYPD,New York City Police Department,Blocked Driveway,11201,BROOKLYN,1008500,181726,...,-73.912589,"(40.665439409429055, -73.91258883811614)",84500,666,67801,588,13905,1034,34398,407.076923
1,19503621,12/31/2010 11:57:30 PM,01/01/2011 12:46:41 AM,NYPD,New York City Police Department,Blocked Driveway,11201,QUEENS,1037441,205363,...,-73.808082,"(40.73019074217415, -73.80808214665396)",42252,14064,3421,17532,6094,930,34398,814.115308
2,19504838,12/31/2010 11:54:59 PM,01/01/2011 08:03:26 AM,NYPD,New York City Police Department,Blocked Driveway,11201,BROOKLYN,1006381,176021,...,-73.920246,"(40.64978599344521, -73.9202456701406)",93877,5066,77175,2319,7358,1396,34398,366.415629
3,19506107,12/31/2010 11:54:00 PM,03/16/2011 09:36:00 AM,DOT,Department of Transportation,Street Light Condition,11201,MANHATTAN,980807,197440,...,-74.012419,"(40.70860327698897, -74.01241861690464)",3011,1947,135,590,248,83,34398,11424.111591
4,19507090,12/31/2010 11:52:00 PM,02/01/2011 12:45:00 PM,DEP,Department of Environmental Protection,Noise,11201,BROOKLYN,978741,169492,...,-74.019848,"(40.63189122768689, -74.01984766770063)",68853,47714,1173,7889,10322,1419,34398,499.586075
